<a href="https://colab.research.google.com/github/Yan344933/PythonPractice/blob/PageRank/assignment2_pagerank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ISOM 3370 Assignment 2  

PageRank is an algorithm used by Google to rank web pages in the search engine results. PageRank is an iterative algorithm which is a perfect fit for Spark. In this question, you will implement PageRank algorithm with PySpark by completing the following codes.

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

!wget -q https://www.dropbox.com/s/dylx9471funotis/network.txt

First, we load the sample network file in Spark RDD, as below.

This file has format: **source_page target_page**. Below is the content of the file.


page1 page3

page2 page1

page4 page1

page3 page1

page4 page2

page3 page4

Run the following cells, and complete the cells.

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

In [ ]:
# load the file and persist the RDD in memory
rdd = sc.textFile('network.txt').persist()

In [ ]:
# this is how we compute contribute score for each link.
# for example, if a page's rank is 1.0 and it has 2 outgoing links, each links will have contribution 0.5.
# (neighbors=[page1, page2], rank=1.0)  => ([(page1, 0.5), (page2,0.5)])
def computeContribs(neighbors, rank):
  for neighbor in neighbors:
      yield (neighbor, rank/len(neighbors))

In [ ]:
# TO COMPLETE: for each page, output its outgoing neighbors in a list.
# For example: page3 has two outgoing links to page 1 and page4, the output will be (page3, [page1, page4])
# The content of linksRDD is:
# page4 [u'page1', u'page2']
# page2 [u'page1']
# page3 [u'page1', u'page4']
# page1 [u'page3']

linksRDD = rdd.map(lambda s: s.split(" ")).reduceByKey(lambda a, b: [a, b]).map(lambda s: (s[0], [s[1]]) if isinstance(s[1], str) else (s[0], s[1]))
#linksRDD.collect()

[('page1', ['page3']),
 ('page2', ['page1']),
 ('page4', ['page1', 'page2']),
 ('page3', ['page1', 'page4'])]

In [ ]:
# TO COMPLETE: for each page, initialize its pagerank score to be 1.0
# The content of ranksRDD is:
# [(u'page4', 1.0), (u'page2', 1.0), (u'page3', 1.0), (u'page1', 1.0)]

ranksRDD = linksRDD.groupByKey().map(lambda s: (s[0], 1.0))
#ranksRDD.collect()

[('page1', 1.0), ('page2', 1.0), ('page4', 1.0), ('page3', 1.0)]

In [ ]:
# TO COMPLETE: PageRank is an iterative algorithm.
# For each iteration, each page's pagerank score will be updated.
# Refer to lecture slides for method details.
for iteration in range(10):
    # calculate the contribution of each page's outgoing link

    contribs = linksRDD.join(ranksRDD).flatMap(lambda s: computeContribs(s[1][0],s[1][1]))
    #contribs.collect()

    # update each page's page rank by summing up all incoming link's contribution
    ranksRDD = contribs.reduceByKey(lambda a, b: a+b)
    #ranksRDD.collect()

[('page4', 0.7109375),
 ('page2', 0.375),
 ('page1', 1.4296875),
 ('page3', 1.484375)]